# Pre_requisite -- login to the model

In [2]:
!git config --global credential.helper store

!huggingface-cli login




    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
The token `Prash_Mistral_7B_token` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /ro

# Step 1: Set Up Google Colab Environment

In [13]:
# This installs necessary libraries to run Mistral-7B-Instruct-v0.3 efficiently.
!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu121

!pip install --upgrade transformers accelerate




Looking in indexes: https://download.pytorch.org/whl/cu121


# Step 2: Load the Model and Tokenizer

In [14]:
# This loads the model with float16 precision to optimize memory usage.
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define model name
model_name = "mistralai/Mistral-7B-Instruct-v0.3"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("Model loaded successfully!")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded successfully!


# Step 3: Trim Conversation History to Prevent Memory Issues

In [15]:
# This function ensures conversation history does not grow uncontrollably,
# preventing infinite loops and memory issues.

MAX_HISTORY_TOKENS = 2048  # Limit conversation history

def trim_history(history):
    """
    Trims conversation history to prevent memory overflow.
    """
    tokens = tokenizer(history, return_tensors="pt").input_ids
    if tokens.shape[1] > MAX_HISTORY_TOKENS:
        tokens = tokens[:, -MAX_HISTORY_TOKENS:]  # Keep recent tokens only
    return tokenizer.decode(tokens[0], skip_special_tokens=True)

# Step 4: Define Response Generation Function

In [16]:
# This function keeps responses concise and prevents infinite loops
# by setting max_length and limiting history tokens.

def generate_response(conversation_history, user_input, max_length=512, temperature=0.7, top_k=50, top_p=0.9):
    """
    Generates a chatbot response while maintaining conversation history.
    """
    conversation_history += f"\nUser: {user_input}\nBot:"
    conversation_history = trim_history(conversation_history)

    input_ids = tokenizer(conversation_history, return_tensors="pt").input_ids.to(model.device)

    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            max_length=max_length,
            do_sample=True,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p
        )

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    conversation_history += f" {response}\n"
    return response, conversation_history

# Step 5: Run the Chatbot in a Loop with Exit Condition

In [12]:
# This allows user interaction while preventing infinite loops by checking for exit conditions.

conversation_history = ""

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Bot: Goodbye!")
        break

    response, conversation_history = generate_response(conversation_history, user_input)
    print(f"Bot: {response}")

You: How do microwave ovens work?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Bot: 
User: How do microwave ovens work?
Bot: Microwave ovens work by using microwave radiation to heat food. When microwaves are directed at food, they cause the water molecules in the food to vibrate, generating heat. This heat cooks the food, making it hot and ready to eat. The microwave oven also has a magnetron, a device that produces microwaves, and a turntable to evenly distribute the microwaves around the food. The oven also has a control panel with settings for time, power level, and cooking options like defrost and reheat. When you press start, the microwave generates microwaves, cooks your food, and turns off when the cooking time is up.
You: Is there any interesting story behind their invention?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Bot: 
User: How do microwave ovens work?
Bot: 
User: How do microwave ovens work?
Bot: Microwave ovens work by using microwave radiation to heat food. When microwaves are directed at food, they cause the water molecules in the food to vibrate, generating heat. This heat cooks the food, making it hot and ready to eat. The microwave oven also has a magnetron, a device that produces microwaves, and a turntable to evenly distribute the microwaves around the food. The oven also has a control panel with settings for time, power level, and cooking options like defrost and reheat. When you press start, the microwave generates microwaves, cooks your food, and turns off when the cooking time is up.

User: Is there any interesting story behind their invention?
Bot: Yes, the invention of the microwave oven is quite interesting. The concept of using microwaves for cooking was first proposed by a physicist named Percy Spencer in 1945. Spencer worked for the Raytheon Corporation, which was involved i